# 阿里云DSW 一键脚本 By bilibili@十字鱼
## 参考感谢——秋葉aaaki，sukaka
- 十字鱼 https://space.bilibili.com/893892

## 1.安装并运行
- 如果启动不成功，请停止后重复运行第1步。

In [ ]:
import os
import time
from concurrent.futures import ProcessPoolExecutor,ThreadPoolExecutor
from huggingface_hub import HfApi, list_repo_files
#安装目录
path = "/mnt/workspace"
#安装网址
url = "https://gitcode.net/overbill1683/stable-diffusion-webui"
url2 = "https://github.com/Akegarasu/lora-scripts"
repo = url.split('/')[-1]
repo2 = url2.split('/')[-1]
#是否重装
reinstall = False
#修改分支
branch = "master"
branch2 = "main"
#SD模型（先换行，再加链接）
model_urls = """
https://liblibai-online.vibrou.com/models/e7e55f800035004cf0ee31d91f874a415a2b64a4.safetensors?attname=%E7%9C%9F%E5%AE%9E%E6%84%9F%E5%BF%85%E5%A4%87%E6%A8%A1%E5%9E%8B%EF%BD%9CChilloutMix_Chilloutmix-Ni-pruned-fp32-fix.safetensors
"""
#unet模型（先换行，再加链接）
unet_urls = """
"""
#VAE模型（先换行，再加链接）
VAE_urls = """
https://modelscope.cn/api/v1/models/dienstag/anything-v4.0/repo?Revision=master&FilePath=anything-v4.0.vae.pt
"""
#lora模型（先换行，再加链接）
lora_urls = """
"""
#ControlNet模型（先换行，再加链接）
ControlNet_urls = """
"""
#扩展插件（先换行，再加链接,不要带.git）
extension_urls = """
https://gitcode.net/ranting8323/a1111-sd-webui-tagcomplete
https://gitcode.net/overbill1683/stable-diffusion-webui-localization-zh_Hans
https://gitcode.net/ranting8323/sd-webui-additional-networks
https://gitcode.net/ranting8323/sd-webui-infinite-image-browsing
https://gitcode.net/ranting8323/stable-diffusion-webui-wd14-tagger
"""
#embedding模型（先换行，再加链接）
embedding_urls = """
"""
#运动lora（先换行，再加链接）
motion_lora_urls = """
"""
#视频文件（先换行，再加链接）
video_urls = """
"""
config_urls = """
https://gitcode.net/Akegarasu/sd-webui-configs/-/raw/master/config.json
"""
#额外文件（先换行，再加链接）（下载到安装目录，方便剪切）
extra_urls = """
"""
#内存优化（不要修改）
def libtcmalloc():
    !apt-get update -qq > /dev/null 2>&1
    !apt-get install -qq -y zip > /dev/null 2>&1
    !apt-get install -qq -y ffmpeg > /dev/null 2>&1
    !apt-get install -qq -y gifsicle > /dev/null 2>&1
    !apt-get install -qq -y libimage-exiftool-perl > /dev/null 2>&1
    !DEBIAN_FRONTEND="noninteractive" apt-get install -y python3-tk > /dev/null 2>&1
    if os.path.exists(f'{path}/temp'):
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        print('内存优化已安装')
    else:
        os.chdir(f'{path}')
        os.makedirs('temp', exist_ok=True)
        os.chdir('temp')
        os.system('wget -q http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb')
        os.system('apt-get install -qq libunwind8-dev -y > /dev/null 2>&1')
        !dpkg -i *.deb > /dev/null 2>&1
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        !rm *.deb > /dev/null 2>&1
        print('内存优化安装完毕')
#安装主体（不要修改）
def install_main():
    if os.path.exists(f'{path}/{repo}'):
        %cd {path}/{repo}
        #!pip install -U pip > /dev/null 2>&1
        #!pip install -r requirements.txt > /dev/null 2>&1
        #!pip install -U ipywidgets > /dev/null 2>&1
        #!pip install pyOpenSSL watchdog simple_lama_inpainting clip-interrogator onnxruntime-gpu > /dev/null 2>&1
        #!pip install bitsandbytes==0.41.1 tensorboard==2.14.1 tensorflow==2.14.0 > /dev/null 2>&1
        print(f'{repo}升级完毕')
    print(f'开始安装{repo2}')
    if os.path.exists(f'{path}/{repo2}'):
        %cd {path}/{repo2}
        print(f'开始安装{repo2}依赖，请耐心等待')
        !pip install -r requirements.txt > /dev/null 2>&1
        !pip install gradio-tunneling > /dev/null 2>&1
        print(f'{repo2}升级完毕')
#下载文件1（不要修改）
def download_1():#插件
    tasks = []
    tasks.extend(download_git(extension_urls,f'{path}/{repo}'))
    with ThreadPoolExecutor() as executor:
        for task in tasks:
            executor.submit(os.system, task)
#下载文件2（不要修改）
def download_2():
    tasks = []
    tasks.extend(download_aria('https://developer.download.nvidia.com/compute/cuda/11.7.1/local_installers/cuda_11.7.1_515.65.01_linux.run',f'{path}/{repo}'))
    tasks.extend(download_aria('https://huggingface.co/Gluttony10/1/resolve/main/whl/torch-2.1.2%2Bcu118-cp310-cp310-linux_x86_64.whl?download=true',f'{path}/{repo}'))
    tasks.extend(download_aria('https://huggingface.co/Gluttony10/1/resolve/main/whl/xformers-0.0.23.post1%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl?download=true',f'{path}/{repo}'))
    tasks.extend(download_aria('https://huggingface.co/Gluttony10/1/resolve/main/whl/torchaudio-2.1.2%2Bcu118-cp310-cp310-linux_x86_64.whl?download=true',f'{path}/{repo}'))
    tasks.extend(download_aria('https://huggingface.co/Gluttony10/1/resolve/main/whl/torchvision-0.16.2%2Bcu118-cp310-cp310-linux_x86_64.whl?download=true',f'{path}/{repo}'))
    tasks.extend(download_aria(config_urls,f'{path}/{repo}'))
    #tasks.extend(download_aria(embedding_urls,f'{path}/{repo}/models/embeddings'))
    tasks.extend(download_aria(model_urls,f'{path}/{repo}/models/Stable-diffusion'))
    #tasks.extend(download_aria(unet_urls,f'{path}/{repo}/models/unet'))
    tasks.extend(download_aria(VAE_urls,f'{path}/{repo}/models/VAE'))
    #tasks.extend(download_aria(lora_urls,f'{path}/{repo}/models/loras'))
    #tasks.extend(download_aria(ControlNet_urls,f'{path}/{repo}/models/controlnet'))
    #tasks.extend(download_aria(video_urls,f'{path}/{repo}/input/video'))
    #if os.path.exists(f'{path}/{repo}/custom_nodes/comfyui-animatediff'):
        #tasks.extend(download_aria('https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_mm.ckpt?download=true',f'{path}/{repo}/custom_nodes/comfyui-animatediff/models'))
        #tasks.extend(download_aria(motion_lora_urls,f'{path}/{repo}/custom_nodes/comfyui-animatediff/loras'))
    #if os.path.exists(f'{path}/{repo}/custom_nodes/ComfyUI-AnimateDiff-Evolved'):
        #tasks.extend(download_aria('https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_mm.ckpt?download=true',f'{path}/{repo}/custom_nodes/ComfyUI-AnimateDiff-Evolved/models'))
        #tasks.extend(download_aria(motion_lora_urls,f'{path}/{repo}/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora'))
    tasks.extend(download_aria('https://download.pytorch.org/models/mobilenet_v2-b0353104.pth','/root/.cache/torch/hub/checkpoints'))
    tasks.extend(download_aria('https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth','/root/.cache/torch/hub/checkpoints'))
    #tasks.extend(download_hf('CIDAS/clipseg-rd64-refined',f'{path}/{repo}/models/clipseg'))
    #tasks.extend(download_aria('https://github.com/enesmsahin/simple-lama-inpainting/releases/download/v0.1.0/big-lama.pt',f'{path}/{repo}/models/lama'))
    #tasks.extend(download_hf('Salesforce/blip-image-captioning-base',f'{path}/{repo}/models/clip_interrogator/Salesforce/blip-image-captioning-base'))
    #tasks.extend(download_hf('succinctly/text2image-prompt-generator',f'{path}/{repo}/models/prompt_generator/text2image-prompt-generator'))
    #tasks.extend(download_hf('Helsinki-NLP/opus-mt-zh-en',f'{path}/{repo}/models/prompt_generator/opus-mt-zh-en'))
    tasks.extend(download_aria(extra_urls,f'{path}'))
    with ThreadPoolExecutor() as executor:
        for task in tasks:
            executor.submit(os.system, task)
    print('文件下载完毕')
#下载git库（不要修改）
def download_git(links,folder):
    tasks = []
    link = links.strip().split("\n")
    for li in link:
        tasks.append(f'git -C {folder} clone {li} --recursive > /dev/null 2>&1')
    for li in link:
        name = li.split('/')[-1]
        tasks.append(f'git -C {folder}/{name} pull > /dev/null 2>&1')
    for li in link:
        name = li.split('/')[-1]
        tasks.append(f'git -C {folder}/{name} submodule init > /dev/null 2>&1')
    for li in link:
        name = li.split('/')[-1]
        tasks.append(f'git -C {folder}/{name} submodule update > /dev/null 2>&1')
    for li in link:
        name = li.split('/')[-1]
        tasks.append(f'git -C {folder}/{name} fetch > /dev/null 2>&1')
    return tasks
#下载模块（不要修改）
def download_aria(links,folder):
    tasks = []
    link = links.strip().split("\n")
    for li in link:
        if "?download=true" in li:
            li = li.replace("?download=true","")
        if "https://mirror.ghproxy.com/https://github.com" in li:
            li=li.replace("https://mirror.ghproxy.com/https://github.com","https://mirror.ghproxy.com/https://github.com")
        elif "https://github.com" in li:
            li=li.replace("https://github.com","https://mirror.ghproxy.com/https://github.com")
        elif "huggingface.co" in li:
            li=li.replace("huggingface.co","hf-mirror.com")
        elif "civitai.com" in li:
            li=li.replace("civitai.com","civitai.sukaka.top")
        fi = li.split('/')[-1]
        if "." in fi:
            tasks.append(f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -t 10 -d {folder} -o {fi} "{li}" > /dev/null 2>&1')
        else:
            tasks.append(f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -t 10 -d {folder} "{li}" > /dev/null 2>&1')
    return tasks
#下载抱脸库（不要修改）
def download_hf(repo,folder):
    hf_api = HfApi(endpoint="https://hf-mirror.com",)
    tasks = []
    while True:
        try:
            files = hf_api.list_repo_files(f'{repo}')
            for fil in ['.gitattributes','README.md']:
                while fil in files:
                    files.remove(fil)
            break
        except:
            pass
    for fi in files:
        li = os.path.join("https://hf-mirror.com", f'{repo}', "resolve/main/", f'{fi}')
        tasks.append(f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -t 10 -d {folder} -o {fi} "{li}" > /dev/null 2>&1')
    return tasks
#主进程（不要修改）
def main():
    time_start = time.time()
    !git config --global url."https://mirror.ghproxy.com/https://github.com".insteadOf "https://github.com"
    !git config --global url."https://mirror.ghproxy.com/https://raw.githubusercontent.com".insteadOf "https://raw.githubusercontent.com"
    print("运行开始")
    !df -hl #查看磁盘
    !nvidia-smi #查看显卡
    %cd {path}
    if reinstall:
        print('旧文件删除中')
        !rm -rf {path}/{repo}
    print(f'开始安装{repo}')
    !git config --global http.postBuffer 2000003072
    !git -C {path} clone {url} --recursive > /dev/null 2>&1
    !git -C {path} clone {url2} --recursive > /dev/null 2>&1
    if os.path.exists(f'{path}/{repo}'):
        !mkdir -p {path}/{repo}/repositories > /dev/null 2>&1
        %cd repositories
        !git clone "https://gitcode.net/overbill1683/stablediffusion" "stable-diffusion-stability-ai"
        !git clone "https://gitcode.net/overbill1683/taming-transformers" "taming-transformers"
        !git clone "https://gitcode.net/overbill1683/k-diffusion" "k-diffusion"
        !git clone "https://gitcode.net/overbill1683/CodeFormer" "CodeFormer"
        !git clone "https://gitcode.net/overbill1683/BLIP" "BLIP"
    if os.path.exists(f'{path}/{repo2}'):
        !mkdir -p {path}/{repo2}/repositories > /dev/null 2>&1
        !git -C {path}/{repo2} checkout {branch2}
        !git -C {path}/{repo2} pull
        !git -C {path}/{repo2} submodule init > /dev/null 2>&1
        !git -C {path}/{repo2} submodule update > /dev/null 2>&1
        print("多进程开始，请耐心等待")
        with ProcessPoolExecutor() as executor:
            futures = []
            for func in [download_1,download_2,install_main,libtcmalloc]:
                futures.append(executor.submit(func))
                time.sleep(10)
            for future in futures:
                future.result()
        %cd {path}/{repo}
        !pip install torch-2.1.2%2Bcu118-cp310-cp310-linux_x86_64.whl > /dev/null 2>&1
        !pip install torchaudio-2.1.2%2Bcu118-cp310-cp310-linux_x86_64.whl > /dev/null 2>&1
        !pip install torchvision-0.16.2%2Bcu118-cp310-cp310-linux_x86_64.whl > /dev/null 2>&1
        !pip install xformers-0.0.23.post1%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl > /dev/null 2>&1
        !sh cuda_11.7.1_515.65.01_linux.run --toolkit --silent
        time_end = time.time()
        print('\033[32m安装耗时:',int((time_end - time_start)/60),'min\033[0m')
    else:
        print('安装失败请重试')

check = False
if __name__ == '__main__':
    main()

## 2.快速启动

In [ ]:
%cd /mnt/workspace/ComfyUI
!HF_ENDPOINT=https://hf-mirror.com python main.py --use-pytorch-cross-attention

## 3.压缩输出并清空
- 停止后再运行

In [ ]:
%cd /mnt/workspace/ComfyUI
!zip -r output.zip output
!mv output.zip ..
!rm -rf output

## 4.启动训练GUI
- 停止后再顺序运行

In [ ]:
#每次启动实例只运行一次 等待出现gradio网址后，再运行下一步
import threading
def start_gradio():
    !gradio-tunneling --port 28000
background_thread = threading.Thread(target=start_gradio)
background_thread.daemon = True
background_thread.start()

In [ ]:
import os
cuda_path = "/usr/local/cuda-11.7/targets/x86_64-linux/lib"
ld_library_path = os.environ.get("LD_LIBRARY_PATH")
ld_library_path = f"{ld_library_path}:{cuda_path}"
os.environ["LD_LIBRARY_PATH"] = ld_library_path
%cd /mnt/workspace/lora-scripts
!pip uninstall tb-nightly tensorboard tensorflow tensorflow-estimator tf-estimator-nightly tf-nightly -y > /dev/null 2>&1
!pip install -r requirements.txt > /dev/null 2>&1
!HF_ENDPOINT=https://hf-mirror.com python gui.py